In [60]:
"""Processes the FETAL .dcm dataset into .npy files."""

import glob, os, sys, shutil
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from lxml import etree

from pprint import pprint
import gdcm
import pylab

import pydicom
from pydicom.data import get_testdata_files
from pydicom.filereader import read_dicomdir

In [73]:
"""The FETAL dataset comes into the following format:
FetalLung/
    FetalLungNormal/ 
        LC20141215330/
            series/
                IM-0182-0001-d.dcm
                ...
        LC20141217317/
            AX_SSFSE_BODY/
                IM-0991-0001-d.dcm
                ...
            COR_SSFSE_BODY/
                IM-0991-0001-d.dcm
                ...
            SAG_SSFSE_BODY/
                IM-0992-0001-d.dcm
                ...
        ...
    FetalLungAbnormal/ 
        patient/
            LC20150128274/
                AX_SSFSExr_BODY/
                    IM-0782-0001-d.dcm
                    ...
                Cor_SSFSExr_BODY/
                    IM-0781-0001-d.dcm
                    ...
                SAG_SSFSExr_BODY/
                    IM-0783-0001-d.dcm
                    ...
            ...

The output .npy files are placed in the following directory:
FETAL/
    processed/
        0-0002.npy
        ...
with each file as [class]-[scan_id].npy where class is 
0 (abnormal) or 1 (normal).
"""

DEBUG = False

# Directory holding the decompressed normal fetal images
normal_directory = '/../home/mazin/FetalLung/FetalLungNormal'

# Directory holding the decompressed abnormal fetal images
abnormal_directory = '/../home/mazin/FetalLung/FetalLungAbnormal/patient'

# Directory output for the the processed .npy files
output_directory = '../../data/FETAL/processed'
if not os.path.exists(output_directory):
    os.mkdir(output_directory)
else:
    print("Warning: output dir {} already exists".format(output_directory))

assert os.path.isdir(normal_directory), "Couldn't find the dataset at {}".format(normal_directory)
assert os.path.isdir(abnormal_directory), "Couldn't find the dataset at {}".format(abnormal_directory)
assert os.path.isdir(output_directory), "Couldn't find the dataset at {}".format(output_directory)

In [62]:
def getPathsForSubjects(class_path):
    subject_paths = glob.glob("%s/*" % class_path)
    subject_ids = [path.split('/')[-1] for path in subject_paths]
    return subject_ids, subject_paths

def getPathsForScans(subject_path):
    scan_paths = glob.glob("%s/*" % subject_path)
    scan_types = [path.split('/')[-1] for path in scan_paths]
    return scan_types, scan_paths

In [63]:
def getUniqueSeriesPaths(scan_path):
    """
    Group together .dcm by their SeriesInstanceUID attribute

    Returns a dict of unique SeriesInstanceUID => [.dcm paths]
    """
    seriesDict = {}
    dicom_paths = glob.glob("%s/*.dcm" % scan_path)
    for dicom_path in dicom_paths:
        dicom_obj = pydicom.read_file(dicom_path)
        if dicom_obj.SeriesInstanceUID in seriesDict:
            seriesDict[dicom_obj.SeriesInstanceUID].append(dicom_path)
        else:
            seriesDict[dicom_obj.SeriesInstanceUID] = [dicom_path]
    return seriesDict

In [64]:
def getSeriesOrientation(series_paths):
    num_dicoms = len(series_paths)
    
    file = series_paths[0]
    r = gdcm.ImageReader()
    r.SetFileName(file)
    if not r.Read():
      sys.exit(1)
    ir = r.GetImage()

    # orientation
    dircos =  ir.GetDirectionCosines()
    t = gdcm.Orientation.GetType(tuple(dircos))
    orientation = gdcm.Orientation.GetLabel(t)
    
    return orientation
    
def load_dicoms(dicom_paths, displacement):
    num_dicoms = len(dicom_paths)

    dcm_dict = get_dcm_dict(dicom_paths[0])
    ref_ds = dcm_dict['ds']

    # Create the structure for the 3-D volume
    data = np.zeros([num_dicoms, ref_ds.Rows, ref_ds.Columns])

    for dicom_path in dicom_paths:
        dcm_dict = get_dcm_dict(dicom_path)
        ds = dcm_dict['ds']
        if ds.Rows != ref_ds.Rows or ds.Columns != ref_ds.Columns:
            print("Invalid scan size: " +  str([ds.Rows, ds.Columns]))
            continue    
        slice_num = int(ds.InstanceNumber) - displacement
        if slice_num >= len(data):
            print("slice_num: ", slice_num )
            slice_num = len(data) - 1
        if dcm_dict['raw'].shape != data[slice_num, :, :].shape:
            print("Incorrect shape: " + str([dcm_dict['raw'].shape, data[slice_num, :, :].shape]))
            continue
        data[slice_num,:,:] = dcm_dict['raw']
        
    data = np.asarray(data, dtype='int16')

    # sliceBitmap = data[10]
    # plt.imshow(sliceBitmap, cmap=cm.Greys_r)
    # plt.show()

    return data

In [71]:
def convertDicomsToMatrix(class_label, series_paths, series_idx):
    """
    Converts and saves the .dcm files to .npy numpy matrix files in the 
    class directory.
    """
    print("Converting dicom %s to numpy" % series_idx)
    output_file_path = getOutputFilePath(class_label, series_idx) # without orientation
    
    # Get the orientation of the .dcm files in the series (COR/AXL/SAG)
    orientation = getSeriesOrientation(series_paths)
    print(orientation)
    
    # Convert to 3D numpy matrix and save to output directory
    numpy_matrix = getNumpyMatrix(series_paths)

    # If the file exists, ask the user whether they want to overwrite.
    # Exit out of the function without saving if they do not want to overwrite.
    if (len(sys.argv) <= 1 or sys.argv[1] != '--overwrite') and os.path.exists(output_file_path):
        try:
            query_question = "File %s already exists. Would you like to overwrite it?" % output_file_path
            if not query_yes_no(query_question):
                return
        except EOFError as e:
            pass
    
    print("Matrix Dimensions: %s" % str(numpy_matrix.shape))
    np.save(output_file_path, numpy_matrix)

    print("Saved %s" % output_file_path)
    print()

def getNumpyMatrix(series_paths):
    """
    Collects .dcm files into a 3D numpy matrix.
    """
    # Get the dicom paths for this scan_id
    displacement, max_frame_num = seriesPathToMinMaxFrameNumber(series_paths)
    if DEBUG:
        print(displacement)
        print(max_frame_num)
        print()

    # raw_matrix is the 3d array of the dicom slices in order
    print(scan_path)
    raw_matrix = load_dicoms(series_paths, displacement)
    return raw_matrix

# def getDicomPaths(scan_path, scan_id):
#     """
#     Looks in the scan directory for all .dcm files.
#     """
#     dicom_paths = glob.glob("%s/IM-%s*.dcm" % (scan_path, scan_id))
#     if len(dicom_paths) == 0:
#         print(scan_path, scan_id)
#     return dicom_paths

def seriesPathToMinMaxFrameNumber(dicom_paths):
    allFrameNumbers = []
    for dicom_path in dicom_paths:
        frameNumber = getFrameNumberForDicomPath(dicom_path)
        allFrameNumbers.append(frameNumber)
    return min(allFrameNumbers), max(allFrameNumbers)

def getFrameNumberForDicomPath(dicom_path):
    dicom_obj = pydicom.read_file(dicom_path)
    if DEBUG:
        print("FrameOfReferenceUID: ", dicom_obj.FrameOfReferenceUID) 
        print("InstanceNumber: ", dicom_obj.InstanceNumber) 
        print("SliceLocation: ", dicom_obj.SliceLocation) 
        print("WindowCenter: ", dicom_obj.WindowCenter) 
        print("WindowWidth: ", dicom_obj.WindowWidth) #ImageOrientationPatient
        print("SeriesUID: ", dicom_obj.SeriesInstanceUID)
        print("ImagePositionPatient: ", dicom_obj.ImagePositionPatient)
    frameNumber = dicom_obj.InstanceNumber
    return frameNumber

def load_dicoms(dicom_paths, displacement):
    num_dicoms = len(dicom_paths)

    dcm_dict = get_dcm_dict(dicom_paths[0])
    ref_ds = dcm_dict['ds']

    # Create the structure for the 3-D volume
    data = np.zeros([num_dicoms, ref_ds.Rows, ref_ds.Columns])

    for dicom_path in dicom_paths:
        dcm_dict = get_dcm_dict(dicom_path)
        ds = dcm_dict['ds']
        if ds.Rows != ref_ds.Rows or ds.Columns != ref_ds.Columns:
            print("Invalid scan size: " +  str([ds.Rows, ds.Columns]))
            continue    
        slice_num = int(ds.InstanceNumber) - displacement
        if slice_num >= len(data):
            print("slice_num: ", slice_num )
            slice_num = len(data) - 1
        if dcm_dict['raw'].shape != data[slice_num, :, :].shape:
            print("Incorrect shape: " + str([dcm_dict['raw'].shape, data[slice_num, :, :].shape]))
            continue
        data[slice_num,:,:] = dcm_dict['raw']
        
    data = np.asarray(data, dtype='int16')

    # sliceBitmap = data[10]
    # plt.imshow(sliceBitmap, cmap=cm.Greys_r)
    # plt.show()

    return data

def get_dcm_dict(dicom_path):
    """
    lung_raw: the raw output from the dicom file
    lung_im: the grayscale array
    ds: the original ds format if you want to work directly with it
    """
    ds = pydicom.read_file(dicom_path)
    lung = ds.pixel_array
    lung_raw = np.asarray(lung, dtype='float')
    lung_im = (lung_raw/lung_raw.max()) * 255.0
    return {'raw': lung_raw, 'grayscale': lung_im, 'ds': ds}

def getOutputFilePath(class_label, series_idx):
    """
    Returns the .npy file name with [label]-[series_idx].npy
    (e.g. 0-1127.npy)
    """
    outputFileName = "%s_%s.npy" % (class_label, str(series_idx).zfill(4))
    outputFilePath = os.path.join(output_directory, outputFileName)
    return outputFilePath

In [67]:
#### Utility Functions ####
def query_yes_no(question, default="yes"):
    valid = {"yes": True, "y": True, "ye": True,
             "no": False, "n": False}
    if default is None:
        prompt = " [y/n] "
    elif default == "yes":
        prompt = " [Y/n] "
    elif default == "no":
        prompt = " [y/N] "
    else:
        raise ValueError("invalid default answer: '%s'" % default)

    while True:
        sys.stdout.write(question + prompt)
        choice = input().lower()
        if default is not None and choice == '':
            return valid[default]
        elif choice in valid:
            return valid[choice]
        else:
            sys.stdout.write("Please respond with 'yes' or 'no' "
                             "(or 'y' or 'n').\n")

In [72]:
class_paths = [normal_directory, abnormal_directory]
series_idx = 0
for class_idx, class_path in enumerate(class_paths): # abnormal, normal
    subject_ids, subject_paths = getPathsForSubjects(class_path) 
    for subject_id, subject_path in zip(subject_ids, subject_paths): 
        scan_types, scan_paths = getPathsForScans(subject_path) # e.g. series/, COR/, AXL/, etc.
        for scan_type, scan_path in zip(scan_types, scan_paths):
            series_dict = getUniqueSeriesPaths(scan_path) # get the unique scans in the class
            for series_id, series_paths in series_dict.items():
                convertDicomsToMatrix(class_idx, series_paths, series_idx)
                series_idx += 1


Converting dicom 0 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC2562148/SAG SSFSE BODY
Matrix Dimensions: (24, 256, 256)
Saved ../../data/FETAL/processed/0_0000.npy

Converting dicom 1 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC2562148/COR SSFSE BODY
Matrix Dimensions: (15, 256, 256)
Saved ../../data/FETAL/processed/0_0001.npy

Converting dicom 2 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC2562148/AX SSFSE BODY
Matrix Dimensions: (27, 256, 256)
Saved ../../data/FETAL/processed/0_0002.npy

Converting dicom 3 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC840004/SAG SSFSE BODY
Matrix Dimensions: (24, 512, 512)
Saved ../../data/FETAL/processed/0_0003.npy

Converting dicom 4 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC840004/COR SSFSE BODY
Matrix Dimensions: (24, 512, 512)
Saved ../../data/FETAL/processed/0_0004.npy

Converting dicom 5 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC840004/AX SSFSE BODY LARGE
Matr

Converting dicom 46 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20160421276/series
Matrix Dimensions: (36, 512, 512)
Saved ../../data/FETAL/processed/0_0046.npy

Converting dicom 47 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20160421276/series
Matrix Dimensions: (50, 512, 512)
Saved ../../data/FETAL/processed/0_0047.npy

Converting dicom 48 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20160421276/series
Matrix Dimensions: (38, 256, 256)
Saved ../../data/FETAL/processed/0_0048.npy

Converting dicom 49 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20150513371/Cor SSFSEx BODY
Matrix Dimensions: (17, 512, 512)
Saved ../../data/FETAL/processed/0_0049.npy

Converting dicom 50 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20150513371/Sag SSFSEx BODY
Matrix Dimensions: (15, 512, 512)
Saved ../../data/FETAL/processed/0_0050.npy

Converting dicom 51 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20150513371/AX SSFSEx BO

Converting dicom 92 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20150225201/COR SSFSExr BODY
Matrix Dimensions: (24, 512, 512)
Saved ../../data/FETAL/processed/0_0092.npy

Converting dicom 93 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20150225201/AX SSFSExr BODY
Matrix Dimensions: (39, 512, 512)
Saved ../../data/FETAL/processed/0_0093.npy

Converting dicom 94 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC2686890/SAG SSFSEx BODY
Matrix Dimensions: (19, 512, 512)
Saved ../../data/FETAL/processed/0_0094.npy

Converting dicom 95 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC2686890/COR SSFSEx BODY
Matrix Dimensions: (19, 512, 512)
Saved ../../data/FETAL/processed/0_0095.npy

Converting dicom 96 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC2686890/AX SSFSEx BODY
Matrix Dimensions: (32, 512, 512)
Saved ../../data/FETAL/processed/0_0096.npy

Converting dicom 97 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC1077409/S

Converting dicom 137 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC2691704/SAG SSFSEx BH
Matrix Dimensions: (25, 512, 512)
Saved ../../data/FETAL/processed/0_0137.npy

Converting dicom 138 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC2691704/COR SSFSEx BH
Matrix Dimensions: (21, 512, 512)
Saved ../../data/FETAL/processed/0_0138.npy

Converting dicom 139 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC2691704/Ax SSFSEx
Matrix Dimensions: (32, 512, 512)
Saved ../../data/FETAL/processed/0_0139.npy

Converting dicom 140 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC2691704/AX SSFSEx BH
Matrix Dimensions: (31, 512, 512)
Saved ../../data/FETAL/processed/0_0140.npy

Converting dicom 141 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC961597/SAG SSFSE BODY
Matrix Dimensions: (21, 512, 512)
Saved ../../data/FETAL/processed/0_0141.npy

Converting dicom 142 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC961597/COR SSFSE BODY
Matr

/../home/mazin/FetalLung/FetalLungNormal/LC20140709164/AX SSFSEx BODY
Matrix Dimensions: (57, 512, 512)
Saved ../../data/FETAL/processed/0_0182.npy

Converting dicom 183 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC2089655/SAG SSFSE BODY
Matrix Dimensions: (21, 512, 512)
Saved ../../data/FETAL/processed/0_0183.npy

Converting dicom 184 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC2089655/COR SSFSE BODY
Matrix Dimensions: (20, 512, 512)
Saved ../../data/FETAL/processed/0_0184.npy

Converting dicom 185 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC2089655/AX SSFSE BODY
Matrix Dimensions: (24, 512, 512)
Saved ../../data/FETAL/processed/0_0185.npy

Converting dicom 186 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20170607382/series
Matrix Dimensions: (22, 512, 512)
Saved ../../data/FETAL/processed/0_0186.npy

Converting dicom 187 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungNormal/LC20170607382/series
Matrix Dimensions: (34, 512, 512)


...

Matrix Dimensions: (22, 512, 512)
Saved ../../data/FETAL/processed/1_4450.npy

Converting dicom 4451 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungAbnormal/patient/LC20150518341/AX SSFSExr BODY
Matrix Dimensions: (40, 512, 512)
Saved ../../data/FETAL/processed/1_4451.npy

Converting dicom 4452 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungAbnormal/patient/LC1183387/SAG SSFSE BODY
Matrix Dimensions: (22, 512, 512)
Saved ../../data/FETAL/processed/1_4452.npy

Converting dicom 4453 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungAbnormal/patient/LC1183387/SAG SSFSE BODY
slice_num:  4
Matrix Dimensions: (3, 512, 512)
Saved ../../data/FETAL/processed/1_4453.npy

Converting dicom 4454 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungAbnormal/patient/LC1183387/AX SSFSE BODY
Matrix Dimensions: (40, 512, 512)
Saved ../../data/FETAL/processed/1_4454.npy

Converting dicom 4455 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungAbnormal/patient/LC692762/SAG SSFSE BODY
Matrix Dimensions: (24, 

Converting dicom 4493 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungAbnormal/patient/LC671102/AX SSFSE BODY
Matrix Dimensions: (34, 512, 512)
Saved ../../data/FETAL/processed/1_4493.npy

Converting dicom 4494 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungAbnormal/patient/LC794437/SAG SSFSE BODY
Matrix Dimensions: (18, 512, 512)
Saved ../../data/FETAL/processed/1_4494.npy

Converting dicom 4495 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungAbnormal/patient/LC794437/COR SSFSE BODY TRUE REPEAT
Matrix Dimensions: (16, 512, 512)
Saved ../../data/FETAL/processed/1_4495.npy

Converting dicom 4496 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungAbnormal/patient/LC794437/AX SSFSE BODY LARGE
Matrix Dimensions: (32, 512, 512)
Saved ../../data/FETAL/processed/1_4496.npy

Converting dicom 4497 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungAbnormal/patient/LC2564263/SAG SSFSE BODY
Matrix Dimensions: (23, 512, 512)
Saved ../../data/FETAL/processed/1_4497.npy

Converting dicom 4498 to num

Matrix Dimensions: (20, 256, 256)
Saved ../../data/FETAL/processed/1_4535.npy

Converting dicom 4536 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungAbnormal/patient/LC439004/AX SSFSE BODY
Matrix Dimensions: (37, 256, 256)
Saved ../../data/FETAL/processed/1_4536.npy

Converting dicom 4537 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungAbnormal/patient/LC655552/SAG SSFSE BODY
Matrix Dimensions: (17, 512, 512)
Saved ../../data/FETAL/processed/1_4537.npy

Converting dicom 4538 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungAbnormal/patient/LC655552/COR SSFSE BODY
Matrix Dimensions: (18, 512, 512)
Saved ../../data/FETAL/processed/1_4538.npy

Converting dicom 4539 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungAbnormal/patient/LC655552/AX SSFSE BODY
Matrix Dimensions: (28, 512, 512)
Saved ../../data/FETAL/processed/1_4539.npy

Converting dicom 4540 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungAbnormal/patient/LC2089683/SAG SSFSE BODY
Matrix Dimensions: (17, 512, 512)
Saved ../../

Converting dicom 4578 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungAbnormal/patient/LC2038328/SAG SSFSE BODY
Matrix Dimensions: (22, 512, 512)
Saved ../../data/FETAL/processed/1_4578.npy

Converting dicom 4579 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungAbnormal/patient/LC2038328/COR SSFSE BODY
Matrix Dimensions: (15, 512, 512)
Saved ../../data/FETAL/processed/1_4579.npy

Converting dicom 4580 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungAbnormal/patient/LC2038328/AX SSFSE BODY REPEAT
Matrix Dimensions: (17, 512, 512)
Saved ../../data/FETAL/processed/1_4580.npy

Converting dicom 4581 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungAbnormal/patient/LC2069901/SAG SSFSE BODY
Matrix Dimensions: (17, 512, 512)
Saved ../../data/FETAL/processed/1_4581.npy

Converting dicom 4582 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungAbnormal/patient/LC2069901/COR SSFSE BODY
Matrix Dimensions: (16, 512, 512)
Saved ../../data/FETAL/processed/1_4582.npy

Converting dicom 4583 to numpy
AXI

Converting dicom 4620 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungAbnormal/patient/LC626539/AX SSFSE BODY
Matrix Dimensions: (22, 512, 512)
Saved ../../data/FETAL/processed/1_4620.npy

Converting dicom 4621 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungAbnormal/patient/LC20170821370/SAG SSFSE BODY
Matrix Dimensions: (17, 512, 512)
Saved ../../data/FETAL/processed/1_4621.npy

Converting dicom 4622 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungAbnormal/patient/LC20170821370/COR SSFSE BODY
Matrix Dimensions: (19, 512, 512)
Saved ../../data/FETAL/processed/1_4622.npy

Converting dicom 4623 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungAbnormal/patient/LC20170821370/AX SSFSE BODY
Matrix Dimensions: (31, 512, 512)
Saved ../../data/FETAL/processed/1_4623.npy

Converting dicom 4624 to numpy
AXIAL
/../home/mazin/FetalLung/FetalLungAbnormal/patient/LC20141231279/Cor SSFSEx body
Matrix Dimensions: (40, 512, 512)
Saved ../../data/FETAL/processed/1_4624.npy

Converting dicom 4625 to n